## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Padang,ID,-0.9492,100.3543,79.09,79,96,3.00,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,46.06,66,20,0.00,few clouds
2,2,Port Blair,IN,11.6667,92.7500,82.44,78,20,2.30,mist
3,3,Rikitea,PF,-23.1203,-134.9692,78.96,74,35,17.98,scattered clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,66.09,81,100,16.11,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum desired temperature for your trip? "))
max_temp = float(input("What is the maximum desired temperature for your trip? "))

What is the minimum desired temperature for your trip? 70
What is the maximum desired temperature for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Padang,ID,-0.9492,100.3543,79.09,79,96,3.00,overcast clouds
2,2,Port Blair,IN,11.6667,92.7500,82.44,78,20,2.30,mist
3,3,Rikitea,PF,-23.1203,-134.9692,78.96,74,35,17.98,scattered clouds
5,5,Kapaa,US,22.0752,-159.3190,73.72,84,100,17.27,overcast clouds
7,7,Saint-Paul,RE,-21.0096,55.2707,80.71,83,30,1.77,light rain
8,8,San Cristobal,VE,7.7669,-72.2250,77.18,35,87,9.44,overcast clouds
9,9,Cidreira,BR,-30.1811,-50.2056,72.57,79,84,9.35,broken clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,82.36,72,100,5.70,overcast clouds
11,11,Carnarvon,AU,-24.8667,113.6333,71.67,73,0,17.27,clear sky
13,13,Cape Town,ZA,-33.9258,18.4232,79.21,49,0,12.66,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Padang,ID,-0.9492,100.3543,79.09,79,96,3.00,overcast clouds
2,2,Port Blair,IN,11.6667,92.7500,82.44,78,20,2.30,mist
3,3,Rikitea,PF,-23.1203,-134.9692,78.96,74,35,17.98,scattered clouds
5,5,Kapaa,US,22.0752,-159.3190,73.72,84,100,17.27,overcast clouds
7,7,Saint-Paul,RE,-21.0096,55.2707,80.71,83,30,1.77,light rain
...,...,...,...,...,...,...,...,...,...,...
671,671,Emerald,AU,-23.5333,148.1667,71.74,83,7,4.61,clear sky
672,672,Bima,ID,-8.4667,118.7167,75.69,94,4,2.62,clear sky
674,674,Bukama,CD,-9.2000,25.8500,77.11,83,73,1.34,light rain
676,676,Santa Cruz,BO,-17.8000,-63.1667,85.46,70,75,17.27,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Padang,ID,79.09,overcast clouds,-0.9492,100.3543,
2,Port Blair,IN,82.44,mist,11.6667,92.7500,
3,Rikitea,PF,78.96,scattered clouds,-23.1203,-134.9692,
5,Kapaa,US,73.72,overcast clouds,22.0752,-159.3190,
7,Saint-Paul,RE,80.71,light rain,-21.0096,55.2707,
8,San Cristobal,VE,77.18,overcast clouds,7.7669,-72.2250,
9,Cidreira,BR,72.57,broken clouds,-30.1811,-50.2056,
10,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,
11,Carnarvon,AU,71.67,clear sky,-24.8667,113.6333,
13,Cape Town,ZA,79.21,clear sky,-33.9258,18.4232,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Padang,ID,79.09,overcast clouds,-0.9492,100.3543,Mercure Padang
2,Port Blair,IN,82.44,mist,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
3,Rikitea,PF,78.96,scattered clouds,-23.1203,-134.9692,People ThankYou
5,Kapaa,US,73.72,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Saint-Paul,RE,80.71,light rain,-21.0096,55.2707,Boucan Canot


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))